## Read in Libraries

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore') 

## Defining import / export file paths

In [4]:
raw_data_path = r'04a. Facebook - Missing values - Raw data.csv'
export_data_path = 'Processed Data/'
file_name = 'final_facebook.csv'

## Read in Facebook/Instagram Spend and Impressions Data

In [6]:
# Read a CSV file from a specified location and store the data in a pandas DataFrame called 'data_for_imputation'.
# It will then print the data types of each column in the DataFrame to help understand the structure of the data.

data_for_imputation = pd.read_csv(raw_data_path)
data_for_imputation.dtypes

week_starting_date     object
fbig_spend            float64
fbig_imp              float64
dtype: object

In [65]:
# Convert the 'week_starting_date' column in the 'data_for_imputation' by dataFrame to a pandas datetime format 
# using the 'pd.to_datetime' method. This conversion is necessary to perform time-based analysis and modeling on the data. 
# After this code is executed, the 'week_starting_date' column will be of datetime type, which will enable time-based operations to be performed on it.

data_for_imputation['week_starting_date'] = pd.to_datetime(data_for_imputation['week_starting_date'])

# The ".head()" method in pandas is used to print the first few rows of a DataFrame.
# By passing the argument "10" within the parenthesis, this code specifies to print the first 10 rows of the DataFrame.

print(data_for_imputation.head(10))


  week_starting_date  fbig_spend  fbig_imp
0         2019-01-07         0.0       0.0
1         2019-01-14         0.0       0.0
2         2019-01-21         0.0       0.0
3         2019-01-28         0.0       0.0
4         2019-02-04         0.0       0.0
5         2019-02-11         0.0       0.0
6         2019-02-18         0.0       0.0
7         2019-02-25         0.0       0.0
8         2019-03-04         0.0       0.0
9         2019-03-11         0.0       0.0


In [66]:
# Filter the 'data_for_imputation' DataFrame and select rows where the 'fbig_spend' column has a value greater than zero 
# The 'fbig_imp' column has a value of zero. 
# The 'loc' method is used to locate the rows in the DataFrame that satisfy the given conditions. 
# The '&' symbol between the two conditions represents the 'and' operator, which means that both conditions must be true for a row to be selected. 
# This code is useful for finding cases where there is spending on Facebook ads, but there are no impressions, which could indicate an issue with the ad campaign or tracking.

data_for_imputation.loc[(data_for_imputation['fbig_spend']>0)&
                                               (data_for_imputation['fbig_imp']==0)]

,week_starting_date,fbig_spend,fbig_imp
21,2019-06-03,13444.237150,0.0
22,2019-06-10,17661.095060,0.0
23,2019-06-17,5969.195235,0.0
24,2019-06-24,6456.339760,0.0


## Use Overall cost per impression to impute missing impression values

In [67]:
# The Python code in the above cell performs the following steps:

# It filters the 'data_for_imputation' DataFrame to select only the rows where there are both Facebook ad impressions and spending. The resulting DataFrame is stored in the 'data_for_imputation_filtered' variable.
# It calculates the cost per impression (CPI) for the filtered data by summing the 'fbig_spend' column and dividing it by the sum of the 'fbig_imp' column. The result is stored in the 'cost_per_imp_for_imputation' variable.
# It prints the CPI for the filtered data to the console.
# This code is useful for calculating the average cost per impression for a Facebook ad campaign, which can help to evaluate the effectiveness of the campaign and make decisions about future ad spend. 

data_for_imputation_filtered = (data_for_imputation.loc[(data_for_imputation['fbig_imp']>0)&
                                                            (data_for_imputation['fbig_spend']>0)])


cost_per_imp_for_imputation = (data_for_imputation_filtered['fbig_spend'].sum()/
             data_for_imputation_filtered['fbig_imp'].sum()
            )

print('Cost per Impression : ' + str(cost_per_imp_for_imputation))

Cost per Impression : 0.006257490152210633


In [69]:
#The code in this cell is intended to create a new column called fbig_imp in the data_for_imputation dataframe.
# The values of the fbig_imp column are calculated using numpy.where() function. 
# If the value in fbig_spend column is greater than 0 and the value in the fbig_imp column is 0, then the value in fbig_spend column is divided by cost_per_imp_for_imputation and used as the value for fbig_imp column. Otherwise, the value in the fbig_imp column is retained. The code then sets the index of the data_for_imputation dataframe to week_starting_date.

data_for_imputation['fbig_imp'] = (np.where((data_for_imputation['fbig_spend']>0)
                                                                          & (data_for_imputation['fbig_imp']==0),
                                                                           data_for_imputation['fbig_spend']/cost_per_imp_for_imputation,
                                                                           data_for_imputation['fbig_imp']
                                                                 )
                                                                 )


data_for_imputation = data_for_imputation.set_index('week_starting_date')

In [7]:
# Select rows from the data_for_imputation DataFrame where the value of the fbig_spend column is greater than 0 and the value of the fbig_imp column is 0. 
# This can be used to identify entries where there was a significant spend on Facebook ads but no corresponding impressions were recorded.

data_for_imputation.loc[(data_for_imputation['fbig_spend']>0)&
                                               (data_for_imputation['fbig_imp']==0)]

,week_starting_date,fbig_spend,fbig_imp
21,6/3/2019,13444.237150,0.0
22,6/10/2019,17661.095060,0.0
23,6/17/2019,5969.195235,0.0
24,6/24/2019,6456.339760,0.0


In [72]:
# #This code is printing the total spend in the "fbig_spend" column of the "data_for_imputation" DataFrame. 
# It does this by using the sum() function to calculate the sum of all values in the "fbig_spend" column and then concatenating it to a string to display the result using the print() function.

print('Total Spend : ' + str(data_for_imputation['fbig_spend'].sum()))

Total Spend : 3271267.472244


In [73]:
# #This code is printing the total spend in the "fbig_imp" column of the "data_for_imputation" DataFrame. 
# It does this by using the sum() function to calculate the sum of all values in the "fbig_imp" column and then concatenating it to a string to display the result using the print() function.

print('Total Impressions : ' + str(data_for_imputation['fbig_imp'].sum()))

Total Impressions : 522776287.7242937


## Export processed data

In [8]:
# This code exports the processed data stored in the data_for_imputation dataframe to a CSV file named final_facebook.csv in the Processed Data/ directory.

data_for_imputation.to_csv(export_data_path + file_name)